In [ ]:
# =============================================================================
# EXPERIMENT: Random Forest Classifier
# Date: 22-11-25 00:24 started
# Description: Random Forest with hyperparameter tuning
# =============================================================================

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import warnings
from datetime import datetime
from pathlib import Path

In [2]:
# =============================================================================
# 1. LOAD PREPROCESSED DATA (no need to preprocess again!)
# =============================================================================
print("Loading preprocessed data...")
X = pd.read_pickle('../../data/processed/X_train_processed.pkl')
y = pd.read_pickle('../../data/processed/y_train.pkl')
X_test = pd.read_pickle('../../data/processed/X_test_processed.pkl')
test_ids = pd.read_pickle('../../data/processed/test_ids.pkl')

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

Loading preprocessed data...
X shape: (20885, 55)
y shape: (20885,)


In [3]:
# =============================================================================
# 2. DEFINE EXPERIMENT PARAMETERS
# =============================================================================
EXPERIMENT_NAME = "random_forest_tuned"
MODEL_DESCRIPTION = "Random Forest with RandomizedSearchCV for hyperparameter tuning"

In [5]:
# =============================================================================
# 3. HYPERPARAMETER TUNING WITH RANDOMIZED SEARCH
# =============================================================================
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold


print(f"\n{'='*70}")
print(f"EXPERIMENT: {EXPERIMENT_NAME}")
print(f"{'='*70}")

# Define parameter distribution
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['sqrt', 'log2', 0.3, 0.5],
    'class_weight': ['balanced', 'balanced_subsample']
}

# Cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Randomized search
print("\nRunning RandomizedSearchCV (trying 30 combinations)...")
print("This may take 10-15 minutes...\n")

random_search = RandomizedSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_distributions=param_dist,
    n_iter=30,  # Try 30 random combinations
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

random_search.fit(X, y)

# =============================================================================
# 4. RESULTS
# =============================================================================
print(f"\n{'='*70}")
print("HYPERPARAMETER TUNING RESULTS")
print(f"{'='*70}")

print(f"\nBest parameters found:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest CV ROC-AUC: {random_search.best_score_:.4f}")

# Show top 5 parameter combinations
results_df = pd.DataFrame(random_search.cv_results_)
top_5 = results_df.nsmallest(5, 'rank_test_score')[
    ['params', 'mean_test_score', 'std_test_score']
].copy()
top_5['mean_test_score'] = top_5['mean_test_score'].round(4)
top_5['std_test_score'] = top_5['std_test_score'].round(4)

print("\nTop 5 parameter combinations:")
print(top_5.to_string(index=False))

# =============================================================================
# 5. TRAIN FINAL MODEL WITH BEST PARAMETERS
# =============================================================================
print(f"\n{'='*70}")
print("TRAINING FINAL MODEL")
print(f"{'='*70}")

# Best model is already trained by RandomizedSearchCV
model = random_search.best_estimator_

# Training performance (sanity check)
y_pred_train = model.predict_proba(X)[:, 1]
train_auc = roc_auc_score(y, y_pred_train)
print(f"\nTraining ROC-AUC: {train_auc:.4f}")

# Cross-validation score (already computed)
print(f"Cross-Validation ROC-AUC: {random_search.best_score_:.4f}")

# =============================================================================
# 6. FEATURE IMPORTANCE ANALYSIS
# =============================================================================
print(f"\n{'='*70}")
print("FEATURE IMPORTANCE ANALYSIS")
print(f"{'='*70}")

# Get feature importances
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 15 most important features:")
print(feature_importance.head(15).to_string(index=False))


EXPERIMENT: random_forest_tuned

Running RandomizedSearchCV (trying 30 combinations)...
This may take 10-15 minutes...

Fitting 5 folds for each of 30 candidates, totalling 150 fits

HYPERPARAMETER TUNING RESULTS

Best parameters found:
  n_estimators: 500
  min_samples_split: 20
  min_samples_leaf: 4
  max_features: sqrt
  max_depth: 20
  class_weight: balanced_subsample

Best CV ROC-AUC: 0.8641

Top 5 parameter combinations:
                                                                                                                                              params  mean_test_score  std_test_score
{'n_estimators': 500, 'min_samples_split': 20, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'class_weight': 'balanced_subsample'}           0.8641          0.0081
 {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 0.3, 'max_depth': None, 'class_weight': 'balanced_subsample'}           0.8636          0.0080
         {'n_estimator

In [6]:
# =============================================================================
# 4. GENERATE PREDICTIONS
# =============================================================================
test_proba = model.predict_proba(X_test)[:, 1]

In [8]:
# =============================================================================
# 5. CREATE SUBMISSION FILE
# =============================================================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
submission_filename = f"../../outputs/predictions/RF/{EXPERIMENT_NAME}_{timestamp}.csv"

submission = pd.DataFrame({
    'icustay_id': test_ids,
    'prediction': test_proba
})
submission.to_csv(submission_filename, index=False)
print(f"\n✓ Submission saved: {submission_filename}")


✓ Submission saved: ../../outputs/predictions/RF/random_forest_tuned_20251122_0043.csv


In [ ]:
# =============================================================================
# 9. LOG EXPERIMENT
# =============================================================================
experiment_log = {
    'timestamp': timestamp,
    'experiment_name': EXPERIMENT_NAME,
    'model_type': 'RandomForestClassifier',
    'cv_roc_auc_mean': random_search.best_score_,
    'cv_roc_auc_std': results_df.loc[random_search.best_index_, 'std_test_score'],
    'train_roc_auc': train_auc,
    'kaggle_public_score': None,  # Fill in manually after submission
    'kaggle_private_score': None,
    'parameters': str(random_search.best_params_),
    'description': MODEL_DESCRIPTION,
    'submission_file': submission_filename
}

# Append to experiment log
log_file = '../experiment_log.csv'
log_df = pd.DataFrame([experiment_log])

import os
if os.path.exists(log_file):
    log_df.to_csv(log_file, mode='a', header=False, index=False)
else:
    log_df.to_csv(log_file, index=False)

print(f"✓ Experiment logged to {log_file}")

✓ Experiment logged to ../experiment_log.csv


In [10]:
# =============================================================================
# 7. SAVE MODEL (OPTIONAL)
# =============================================================================
import pickle


model_filename = f"../../outputs/models/{EXPERIMENT_NAME}_{timestamp}.pkl"
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)
print(f"✓ Model saved: {model_filename}")

print(f"\n{'='*70}")
print("EXPERIMENT COMPLETE!")
print(f"{'='*70}")
print(f"\nNext steps:")
print(f"1. Upload {submission_filename} to Kaggle")
print(f"2. Record Kaggle score in {log_file}")
print(f"3. Copy this notebook and modify for next experiment")

✓ Model saved: ../../outputs/models/random_forest_tuned_20251122_0043.pkl

EXPERIMENT COMPLETE!

Next steps:
1. Upload ../../outputs/predictions/RF/random_forest_tuned_20251122_0043.csv to Kaggle
2. Record Kaggle score in ../experiment_log.csv
3. Copy this notebook and modify for next experiment
